In [1]:
import torch
import numpy as np
import k3d

from libs import tpcpath
from torchpcp.utils import pytorch_tools
pytorch_tools.set_seed(0)
device = pytorch_tools.select_device("cuda")


In [2]:
def visualize_pc(pc, colors=None):
    if colors is None:
        colors = np.full(len(pc), 0x000000, dtype=np.float64) # white: 0xFFFFFF
    plot = k3d.plot()
    points = k3d.points(pc, colors.astype(np.float32), point_size=0.04, shader='flat')
    plot += points
    plot.display()


In [3]:
from torchpcp.datasets.PointNet2.ModelNet import ModelNet40

In [4]:
path = "/mnt/databox/datasets/ModelNet/modelnet40_normal_resampled"
datasets = ModelNet40(path)

fl: 100%|██████████| 2468/2468 [02:24<00:00, 17.11it/s]


In [12]:
import h5py
# from tqdm import tqdm

def create_modelnet_dataset(file_name, dataset):
    with h5py.File(file_name, "w") as f:
        point_clouds = []
        labels = []
        for i in range(len(dataset)):
            points, label = dataset[i]
            point_clouds.append(points)
            labels.append(label)

        f.create_dataset("point_clouds", data=point_clouds, compression='gzip', compression_opts=1, dtype="float32")
        f.create_dataset("labels", data=labels, compression='gzip', compression_opts=1, dtype="uint8")

create_modelnet_dataset("train_modelnet40.h5", datasets["train"])
create_modelnet_dataset("test_modelnet40.h5", datasets["test"])


In [17]:
from torch.utils.data import Dataset
import os

class MyModelNet40Dataset(Dataset):
    def __init__(self, root, split):
        self.root = root
        if split == "train":
            path = os.path.join(self.root, "train_modelnet40.h5")
        elif split == "test":
            path = os.path.join(self.root, "test_modelnet40.h5")
        else:
            raise NotImplementedError()
            
        with h5py.File(path, "r") as f:
            self.point_clouds = f['point_clouds'][:]
            self.labels = f['labels'][:]
    
    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, idx):
        point_cloud = self.point_clouds[idx]
        label = self.labels[idx]
        return point_cloud, label

my_dataset = MyModelNet40Dataset("./", "train")
points, label = my_dataset[0]
    

In [ ]:
points, label = datasets["train"][0]

In [18]:
visualize_pc(points[:, :3])

/home/coder/anaconda3/envs/py37/lib/python3.7/site-packages/traittypes/traittypes.py:101: UserWarning: Given trait value dtype "float32" does not match required type "uint32". A coerced copy has been created.
  np.dtype(self.dtype).name))


Output()

In [ ]:
visualize_pc(points[:, 3:6])